## Notebook for spike sorting from .rhd data without using the Pipeline of mountainsort (mac channel has no pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import intan2kwik

# mountainlab imports
from mountainlab_pytools import mlproc as mlp
from mountainlab_pytools import mdaio

from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from importlib import reload

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.info('Logger set')

/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
2018-11-22 20:33:17,899 - root - INFO - Logger set


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [3]:
exp_base = os.path.abspath('/Users/zeke/experiment/')

file_sys = {'exp_base': os.path.abspath('/Users/zeke/experiment/')}

sess_par = {'bird': 'o3g3',
           'sess': 'awake_2017-09-08_2925',
           'rec': 0}

file_sys_defaults = {'exp_base': os.path.abspath('/data/experiment')}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [4]:
raw_folder = os.path.join(exp_base, 'raw_data', sess_par['bird'], 'neural', sess_par['sess'])
glob.glob(os.path.join(raw_folder, '*.rhd'))

[]

In [5]:
raw_folder

'/Users/zeke/experiment/raw_data/o3g3/neural/awake_2017-09-08_2925'

In [6]:
rhd_files = glob.glob(os.path.join(raw_folder, '*.rhd'))
rhd_files.sort()
pd_f = pd.DataFrame(rhd_files, columns=['f_path'])

pd_f['stamp'] = pd_f['f_path'].apply(lambda x: int(os.path.split(x)[-1].split('.')[0].split('_')[-1]))

# Select some of the files
sess_start = 0
sess_end = 144400
pd_session = pd_f.loc[(pd_f['stamp'] > sess_start) & (pd_f['stamp'] < sess_end), :]
pd_session.index.size

0

### select subset of files

In [6]:
rhd_list = pd_session['f_path'].values.tolist()[:2]

In [7]:
reload(intan)

<module 'pipefinch.neural.convert.intan' from '/Users/zeke/repos/pipefinch/pipefinch/neural/convert/intan.py'>

## Scripts for sorting

In [8]:
## Dirty import of sorting helper functions from mountainsort exapmles
# import sys
# example_dir = os.path.abspath('/Users/zeke/repos/mountainlab-js/OLD/examples/spike_sorting/python_example/')
# sys.path.append(example_dir)

# import mountainsort4_v1 as ms4

### prep the files with their nice formats, locations and names


In [9]:
rhd_list

[]

In [13]:
ss_folder = os.path.join('/Users/zeke/experiment/ss_data/', sess_par['bird'], sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)

#todo:
# meta and pars from intan files et al
s_f = 20000
ds_params = {'samplerate': s_f}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))
    
# create the mda file from the rhd_list
#intan_meta, samples_in = intan.intan_to_bin(rhd_list, file_paths['mda'], variant='mda')


2018-09-20 10:00:08,820 - root - INFO - Created session par files /Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/params.json


In [15]:
file_paths

{'mda': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda',
 'params': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/params.json',
 'geom': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/geom.csv',
 'filt': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings.mda',
 'firings_curated': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json'}

In [11]:
msc.sort_dataset?

Signature: msc.sort_dataset(*, file_paths:dict, freq_min:int=300, freq_max:int=6000, adjacency_radius:int=1, detect_threshold:float=3, opts:dict={})
Docstring: <no docstring>
File:      ~/repos/pipefinch/pipefinch/neural/sort/mountain/core.py
Type:      function


In [12]:
reload(msc)

<module 'pipefinch.neural.sort.mountain.core' from '/Users/zeke/repos/pipefinch/pipefinch/neural/sort/mountain/core.py'>

In [16]:
msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1,detect_threshold=3)

2018-09-20 10:01:00,905 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:20000 --outputs timeseries_out:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv


2018-09-20 10:01:02,052 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ephys.bandpass_filter already completed. ]
[ Creating output prv for timeseries_out ... ]
[ Done. ]
RUNNING: ml-run-process ephys.whiten --inputs timeseries:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv --parameters --outputs timeseries_out:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv


2018-09-20 10:01:02,908 - pipefinch.sort.mountain.comre - INFO - Sorting


...

RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:1 detect_threshold:3 --outputs firings_out:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings.mda


2018-09-20 10:03:09,877 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/bin/python3 /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --timeseries=/tmp/mountainlab-tmp/output_f1c7be6f4f80d8558aa8234895e7d8b9883763b8_timeseries_out.mda --geom= --firings_out=/tmp/mountainlab-tmp/tempdir_8f65c43edb_oe0HbU/output_firings_out.mda --adjacency_radius=-1 --detect_sign=1 --detect_threshold=3 --_tempdir=/tmp/mountainlab-tmp/tempdir_8f65c43edb_oe0HbU
Using tempdir=/tmp/mountainlab-tmp/tempdir_8f65c43edb_oe0HbU
Preparing /tmp/mountainlab-tmp/tempdir_8f65c43edb_oe0HbU/timeseries.hdf5...
Preparing neighborhood sorte

2018-09-20 10:03:19,349 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_cluster_metrics_out_cca4e00273c205225f4163565edbe86dde051ae7.prv ...
Locating /tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_metrics_out_32a9adff8caa2493cc426091ccb7df800ae1341b.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --metrics_list=/tmp/mountainlab-tmp/output_71edb2098306b7f60dcf711b70d87c4c04e41468_cluster_metrics_out --metrics_list=/tmp/mountainlab-tmp/output_a57f0cb3bb8ebf675608ea8ffebaed276662d7c2_metrics_out --metrics_out=/tmp/mountainlab-tmp/tempdir_e68272811b_mMjbkC/output_metrics_out.json --_tempdir=/tmp/mountainlab-tmp/tempdir_e68272811b_mMjbkC
Elapsed time for pr

In [ ]:
qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json

In [28]:
p = subprocess.Popen(['ls', '-a'], stdout=subprocess.PIPE)
p.communicate()

(None, None)

In [33]:
file_paths

{'mda': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda',
 'params': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/params.json',
 'geom': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/geom.csv',
 'filt': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings.mda',
 'firings_curated': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json'}

In [17]:
show_args_dict = {'raw': file_paths['mda'], 
             'filt': file_paths['filt'],
             'pre': file_paths['pre'],
            'samplereate': ds_params['samplerate'],
            'firings': file_paths['firings'],
            'cluster_metrics': file_paths['cluster_metrics']}

cmd_args = ['--{} {}'.format(k, v) for k, v in show_args_dict.items()]
cmd = 'qt-mountainview'

#p = subprocess.Popen([cmd, ' '.join(cmd_args)], stdout=subprocess.PIPE)


In [18]:
' '.join([cmd] + cmd_args)

'qt-mountainview --raw /Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda --filt /Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv --pre /Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv --samplereate 20000 --firings /Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings.mda --cluster_metrics /Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json'

In [29]:
import subprocess
p = subprocess.Popen(['qt-mountainview', 
                      '--raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json'],
                    stdout=subprocess.PIPE)


In [36]:
# Bandpass
mlp.runProcess('ephys.bandpass_filter', 
               {'timeseries': file_paths['mda']}, {'timeseries_out': file_paths['filt']}, 
               {'samplerate': 20000, 'freq_min': 300, 'freq_max': 6000})



RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:20000 --outputs timeseries_out:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/filt.mda
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ephys.bandpass_filter already completed. ]
[ Done. ]


{'timeseries_out': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/filt.mda'}

In [29]:
## visualize the time series
os.system('ev-timeseries --timeseries{}'.format(file_paths['mda']))

0

In [19]:
ms4.read_dataset_params(ss_folder)

NameError: name 'ms4' is not defined

In [56]:
from importlib import reload
reload(ms4)

<module 'mountainsort4_v1' from '/Users/zeke/repos/mountainlab-js/OLD/examples/spike_sorting/python_example/mountainsort4_v1.py'>

In [49]:
ms4.sort_dataset(dataset_dir=ss_folder,output_dir=out_folder,adjacency_radius=-1,detect_threshold=3)

RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:20000 --outputs timeseries_out:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ephys.bandpass_filter already completed. ]
[ Creating output prv for timeseries_out ... ]
[ Done. ]
RUNNING: ml-run-process ephys.whiten --inputs timeseries:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv --parameters --outputs timeseries_out:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv ...
[ Comp

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [47]:
out_folder

'/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out'

In [57]:
ms4.automated_curation(firings=file_paths['firings'], 
                       cluster_metrics=file_paths['cluster_metrics'],
                       firings_out=file_paths['firings_curated'],
                       opts={})

RUNNING: ml-run-process ms4alg.create_label_map --inputs metrics:/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json --parameters --outputs label_map_out:/tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_label_map_out_eb92531139390e974bf4b9d339f8c7e18b564a6c.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/bin/python3 /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ml_ms4alg/curation_spec.py.mp ms4alg.create_label_map --metrics=/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json --label_map_out=/tmp/mountainlab-tmp/tempdir_e2c7d0d714_FtfUNT/output_label_map_out --_tempdir=/tmp/mountainlab-tmp/tempdir_e2c7d0d714_FtfUNT
Elapsed t

{'firings_out': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings_curated.mda'}

In [53]:
file_paths

{'mda': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda',
 'params': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/params.json',
 'geom': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/geom.csv',
 'filt': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings.mda',
 'firigs_curated': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json'}

In [ ]:
ms4.compute_cluster_metrics(*,timeseries,firings,metrics_out,samplerate,opts={})

In [30]:
output_dir = os.path.join(ss_folder, 'sort_out')
with Pipeline:
    ms4.sort_dataset(dataset_dir=ss_folder,output_dir=output_dir,adjacency_radius=-1,detect_threshold=3)

Output()

Finished pipeline.


In [ ]:
# viewing examples
ev-templates output/templates.mda.prv
ev-timeseries dataset/raw.mda.prv --firings output/firings.mda.prv --samplerate=30000
qt-mountainview --raw dataset/raw.mda.prv --filt output/filt.mda.prv --pre output/pre.mda.prv --samplerate 30000 --firings output/firings.mda.prv
